In [1]:
import pandas
import sys
import os
import json
sys.path.insert(0, os.getcwd())
from data_prep_helpers import recursive_find, read_json

In [2]:
# Read in the experiment json files. One experiment means one package at once version
# across splices (versions of a single dependencny) and predictors
experiments = list(recursive_find("results/artifacts/results"))
print('Found %s experiments.' % len(experiments))

Found 1846 experiments.


In [3]:
# This will populate the data into a data frame based on overall package, splices, and results
df = pandas.DataFrame(columns=["package", "dependency", "result", "success"])
for e in experiments:
    data = read_json(e)
    for res in data:
        df.loc[len(df.index),:] = [res['package'], res['splice'], res['result'], res['success']]

In [4]:
# What are all the splices we tried, where an experiment is a package and version, and dependency at version?
df

,package,dependency,result,success
0,aml@0.1.0,aml@0.1.0,package-install-failed,False
1,warpx@21.12,fftw@2.1.5,splice-failed,False
2,warpx@21.12,fftw@3.3.10,splice-success,True
3,warpx@21.12,fftw@3.3.4,splice-success,True
4,warpx@21.12,fftw@3.3.5,splice-success,True
...,...,...,...,...
5233,raja@0.4.1,raja@0.4.1,package-install-failed,False
5234,raja@0.4.1,raja@0.4.1,package-install-failed,False
5235,raja@0.5.1,raja@0.5.1,package-install-failed,False
5236,raja@0.5.1,raja@0.5.1,package-install-failed,False


In [5]:
# What are the overall statuses of splicing? We can only use successful splices
# This is data for Table 2
pandas.DataFrame(df.groupby("result")['success'].count())

,success
result,
package-install-failed,759
rewiring-failed,219
splice-concretization-failed,24
splice-failed,808
splice-install-failed,729
splice-success,2699


In [6]:
# Create a table with package, splice, libraries/binaries, predictions and outcome
from data_prep_helpers import create_results_table, create_sizes_table
df = create_results_table(experiments)

# At this point we probably want to count the orginal spack-test that don't work and remove them. The spliced
# result only has meaning given the original is known to work!
df.to_csv('splice-rows-v2.csv')

# Also create sizes so we can say soemething about size/times
sizes = create_sizes_table(experiments)
sizes.to_csv("splice-sizes-v2.csv")

df

,package,dependency,original_lib,spliced_lib,binary,prediction,predictor,seconds,command
0,warpx@21.12,fftw@3.3.10,not used,not used,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,True,binary-checks-symbol-provisioner-change,0.001109,binary-checks-symbol-provisioner-change
1,warpx@21.12,fftw@3.3.10,not used,not used,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,True,binary-checks-symbol-provisioner-change,0.001618,binary-checks-symbol-provisioner-change
2,warpx@21.12,fftw@3.3.10,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,True,missing-previously-found-symbols,0.000213,missing-previously-found-symbols
3,warpx@21.12,fftw@3.3.10,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,True,missing-previously-found-symbols,0.138369,missing-previously-found-symbols
4,warpx@21.12,fftw@3.3.10,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,True,missing-previously-found-symbols,0.000909,missing-previously-found-symbols
...,...,...,...,...,...,...,...,...,...
158757,raja@0.12.0,camp@0.3.0,not used,not used,not used,True,spack-test-spliced,0.716046,/spack/bin/spack test run /op6nc2
158758,raja@0.12.0,camp@main,not used,not used,not used,False,smeagle,0.412513,unknown
158759,raja@0.12.0,camp@main,not used,not used,/spack/opt/spack/linux-ubuntu22.04-x86_64_v4/g...,True,binary-checks-symbol-provisioner-change,0.000097,binary-checks-symbol-provisioner-change
158760,raja@0.12.0,camp@main,not used,not used,not used,True,spack-test-original,30.58257,/spack/bin/spack test run /uiiafz
